In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer
from utils.utils import get_game_name_and_scores

from models.ncf_model import NCFModel

In [3]:
# Test Fine Tuning on live data
model = NCFModel()

data_loader = DataLoader.load_from_file('test_evaluation_data_loader', use_published_models_path=True, load_live_data_loader=True)
model.load('evaluation_test_ncf_low_weight_decay_increased_lr_best_model_bugfix', load_published_model=True)
# data_loader = DataLoader.load_from_file('test_evaluation_data_loader_embed_all_except_tags_genres', use_published_models_path=True, load_live_data_loader=True)
# model.load('evaluation_test_gcf_embed_all_except_tags_genres_best_model_bugfix_clip_embeddings', load_published_model=True)

model.new_seed(None)
model.set_data_loader(data_loader)
user_to_fine_tune = 76561198103368250 # 76561198166465514 # 76561198835352289 #  #  #
print('Before', model.get_score_between_user_and_game(76561198880605436, 367520))
# model.ncf.add_new_user()
# model.user_to_index[user_to_fine_tune] = len(model.user_nodes)
# model.user_nodes.append(user_to_fine_tune)
# display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
# display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))
model.fine_tune(user_to_fine_tune, debug=False)
print('After', model.get_score_between_user_and_game(76561198880605436, 367520))
print('Fine Tuned User Output')
print(model.get_score_between_user_and_game(user_to_fine_tune, 367520))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))

Before 0.6973773837089539
After 0.6973773837089539
Fine Tuned User Output
0.6052209734916687


,id,name,score
0,1811260,EA SPORTS™ FIFA 23,0.651262
1,892420,懒人修仙传,0.644149
2,1267910,Melvor Idle,0.634868
3,1418630,Dread Hunger,0.632838
4,1687950,Persona 5 Royal,0.630816
5,872410,ROMANCE OF THE THREE KINGDOMS XIV,0.629838
6,529340,Victoria 3,0.629512
7,899770,Last Epoch,0.628780
8,1468810,鬼谷八荒 Tale of Immortal,0.627349
9,1718570,ASTLIBRA Revision,0.626752


,id,name,score
0,298360,Immortal Defense,0.605086
1,942850,I Walk Among Zombies Vol. 1 (Adult Version),0.605082
2,866510,!AnyWay!,0.605080
3,601840,Griftlands,0.605078
4,33110,Alien Shooter: Revisited,0.605076
5,1129250,NEKO-NIN exHeart 3,0.605075
6,795510,Koihime Enbu RyoRaiRai,0.605075
7,393380,Squad,0.605067
8,671860,BattleBit Remastered,0.605063
9,102840,Shank 2,0.605059


In [2]:
# Test cold fine tune eval on a user
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], game_embeddings=['name'])
data_loader.load_random_user_train_tune_test_split(train_user_percentage=0.8, test_user_percentage=0.2, fine_tune_edge_percentage=0.8, test_edge_percentage=0.2, seed=0)

In [2]:
# Test cold fine tune eval on a user
import pandas as pd
model = NCFModel()
model.load('evaluation_cold_fine_tune_test_cf', load_published_model=False)
model.new_seed(None)
model.set_data_loader(data_loader)
# user_to_fine_tune = 76561198021788348 # 5 tune 2 test
# user_to_fine_tune = 76561199061878606 # 18 tune 2 test
# user_to_fine_tune = 76561198255585338 # 41 tune 12 test
# user_to_fine_tune = 76561198817614482 # 49 tune 11 test
# user_to_fine_tune = 76561198202035709 # 259 tune 69 test
for user_to_fine_tune in [76561198021788348, 76561199061878606, 76561198255585338, 76561198817614482, 76561198202035709]:
    fake_interactions = pd.DataFrame(columns=model.data_loader.users_games_df.columns).astype(model.data_loader.users_games_df.dtypes)
    users_games_df = model.data_loader.users_games_df_grouped_by_user.get_group(user_to_fine_tune)
    all_users_games_df_without_test = users_games_df[users_games_df['data_split'] != 'test']
    new_users_games_df = users_games_df[users_games_df['data_split'] == 'tune']
    model._fine_tune(user_to_fine_tune, new_users_games_df, fake_interactions, all_users_games_df_without_test, fake_interactions, debug=True)
    print('Fine Tuned User Output')
    display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
    display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))

NameError: name 'data_loader' is not defined

In [ ]:
# data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], user_game_edge_embeddings = ['playtime_forever'], game_embeddings=['name'])
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers', 'name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)
data_loader.save_data_loader_parameters('test_ncf_data_loader', overwrite=True)

In [ ]:
# base collaborative filter test
save_file_name = 'test_cf_model'
cf_model = NCFModel(num_epochs = 0, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
cf_model.set_data_loader(data_loader)
cf_model.train(debug=True)
cf_model.save(save_file_name, overwrite=True)
print(cf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(cf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(cf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, cf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, cf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# generalized collaborative filter test
save_file_name = 'test_gcf_model'
gcf_model = NCFModel(num_epochs = 0, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
gcf_model.set_data_loader(data_loader)
gcf_model.train(debug=True)
gcf_model.save(save_file_name, overwrite=True)
print(gcf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(gcf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(gcf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, gcf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, gcf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# mlp test
save_file_name = 'test_mlp_model'
mlp_model = NCFModel(num_epochs = 0, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
mlp_model.set_data_loader(data_loader)
mlp_model.train(debug=True)
mlp_model.save(save_file_name, overwrite=True)
print(mlp_model.get_score_between_user_and_game(76561198835352289, 967050))
print(mlp_model.get_score_between_user_and_game(76561198835352289, 578080))
print(mlp_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, mlp_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, mlp_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# ncf test
save_file_name = 'test_ncf_model'
ncf_model = NCFModel(num_epochs = 0, embedding_size = 24, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
ncf_model.set_data_loader(data_loader)
ncf_model.train(debug=True)
ncf_model.save(save_file_name, overwrite=True)
print(ncf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(ncf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(ncf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, ncf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, ncf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# data_loader = DataLoader.load_from_file('test_evaluation_data_loader_embed_all_except_tags', use_published_models_path=False, load_live_data_loader=True)
model = NCFModel()
model.load('evaluation_test_cf_embed_all_except_tags', load_published_model=False)
model.set_data_loader(data_loader)

In [ ]:
# test_user = 76561199048589805 # Only 1 game
test_user = 76561198835352289 # Only 4 games
# test_user = 76561198880605436 # ~20 games
# test_user = 76561199040442492 # ~400 games
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[-10:]))